In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

#%matplotlib inline

# Cargo CSV

In [21]:
test_final = pd.read_csv('../data/test_final_100k.csv')
test_final = test_final.drop('id',axis=1)

In [41]:
test_final.head()

,idaviso,idpostulante
0,739260,6M9ZQR
1,739260,6v1xdL
2,739260,ezRKm9
3,758580,1Q35ej
4,758580,EAN4J6


In [22]:
test_final['idaviso'] = test_final['idaviso'].astype(str) + test_final['idpostulante'].astype(str)

In [23]:
postulantes_educacion = pd.read_csv('../data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_educacion = postulantes_educacion.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv'))
postulantes_educacion = postulantes_educacion.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_1_postulantes_educacion.csv'))
postulantes_educacion = postulantes_educacion.reset_index().drop('index',axis=1)

In [44]:
postulantes_educacion.head(1)

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso


In [45]:
postulantes_educacion.estado.value_counts()

Graduado      721794
En Curso      286282
Abandonado     95117
Name: estado, dtype: int64

In [46]:
postulantes_educacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103193 entries, 0 to 1103192
Data columns (total 3 columns):
idpostulante    1103193 non-null object
nombre          1103193 non-null object
estado          1103193 non-null object
dtypes: object(3)
memory usage: 25.3+ MB


In [24]:
postulantes_genero_y_edad = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [8]:
postulantes_genero_y_edad.head(1)

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM


In [ ]:
vistas = pd.read_csv('../data/datos_navent_fiuba/fiuba_3_vistas.csv')
vistas = vistas.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.reset_index().drop(columns='index')

In [25]:
postulaciones = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')
postulaciones = postulaciones.reset_index().drop('index',axis=1)

In [ ]:
avisos_online = pd.read_csv('../data/datos_navent_fiuba/fiuba_5_avisos_online.csv')
avisos_online = avisos_online.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv'))
# no tiene avisos online pd.read_csv('data/fiuba_desde_15_abril/')
avisos_online = avisos_online.reset_index().drop(columns='index')

In [26]:
avisos_detalle = pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

# Preproceso datos

### Limpio el DF *postulantes_educacion*

In [ ]:
postulantes_educacion.head()

In [ ]:
postulantes_educacion.shape

In [ ]:
postulantes_educacion.isnull().sum()

In [27]:
postulantes_educacion[(postulantes_educacion.duplicated()) & (postulantes_educacion['idpostulante'] == 'NdJl')]

,idpostulante,nombre,estado
298232,NdJl,Posgrado,En Curso
299293,NdJl,Universitario,Graduado
706921,NdJl,Posgrado,En Curso
707427,NdJl,Universitario,Graduado


In [28]:
postulantes_educacion = postulantes_educacion.drop_duplicates(keep='last')

In [29]:
postulantes_educacion = postulantes_educacion.reset_index().drop('index',axis=1)

### Limpio el DF *postulantes_genero_y_edad*

In [5]:
postulantes_genero_y_edad.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [10]:
postulantes_genero_y_edad.shape

(780020, 3)

In [10]:
postulantes_genero_y_edad.isnull().sum()

idpostulante           0
fechanacimiento    34273
sexo                   0
dtype: int64

Luego veremos que hacer con los ids sin fecha de nacimiento, por el momento los dejamos.

In [30]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()

In [31]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zP6MJa']

,idpostulante,fechanacimiento,sexo
168364,0zP6MJa,1985-08-16,FEM
525452,0zP6MJa,1989-08-16,FEM


In [32]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zPvY49']

,idpostulante,fechanacimiento,sexo
485636,0zPvY49,1991-03-26,MASC
678829,0zPvY49,1991-03-26,FEM


In [33]:
#Dejo la ultima aparicion del duplicado, supongo que es el update más reciente
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')

In [34]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zP6MJa']

,idpostulante,fechanacimiento,sexo
525452,0zP6MJa,1989-08-16,FEM


In [35]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zPvY49']

,idpostulante,fechanacimiento,sexo
678829,0zPvY49,1991-03-26,FEM


In [36]:
postulantes_genero_y_edad['sexo'].value_counts()

FEM           251205
MASC          227853
NO_DECLARA     25342
0.0                7
Name: sexo, dtype: int64

Al igual que con la fecha de nacimiento, por el momento, dejamos los ids con sexo 0.0

In [37]:
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

### Limpio el DF *vistas*

In [ ]:
vistas.head()

In [ ]:
vistas.shape

In [ ]:
vistas.isna().sum()

In [ ]:
vistas.isnull().sum()

In [ ]:
vistas = vistas.drop_duplicates(keep='last')

In [ ]:
vistas = vistas.reset_index().drop(columns='index')

### Limpio el DF *postulaciones*

In [23]:
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [20]:
postulaciones.shape

(8311264, 3)

In [21]:
postulaciones.isnull().sum()

idaviso             0
idpostulante        0
fechapostulacion    0
dtype: int64

In [38]:
postulaciones = postulaciones.drop_duplicates(keep='last')

In [39]:
postulaciones = postulaciones.reset_index().drop('index',axis=1)

### Limpio el DF *avisos_online*

In [ ]:
avisos_online.head()

In [ ]:
avisos_online.shape

In [ ]:
avisos_online.isna().sum()

In [ ]:
avisos_online.isnull().sum()

In [63]:
avisos_online = avisos_online.drop_duplicates(keep='last')

NameError: name 'avisos_online' is not defined

In [ ]:
avisos_online = avisos_online.reset_index().drop(columns='index')

### Limpio el DF *avisos_detalle*

In [29]:
avisos_detalle.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [40]:
avisos_detalle.shape

(45969, 11)

In [ ]:
avisos_detalle['idaviso'].duplicated().sum()

In [41]:
avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')

In [42]:
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

# Preprocesamiento

In [43]:
# Agrego la edad de los postulantes
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

In [44]:
postulantes_genero_y_edad.head()

,idpostulante,fechanacimiento,sexo,edad
0,NM5M,1970-12-03,FEM,48.0
1,5awk,1962-12-04,FEM,56.0
2,ZaO5,1978-08-10,FEM,40.0
3,NdJl,1969-05-09,MASC,49.0
4,eo2p,1981-02-16,MASC,37.0


In [32]:
# Despues veo que hago con los que no tienen edad
postulantes_genero_y_edad.isnull().sum()

idpostulante           0
fechanacimiento    26096
sexo                   0
edad               26096
dtype: int64

In [29]:
postulantes_genero_y_edad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504407 entries, 0 to 504406
Data columns (total 4 columns):
idpostulante       504407 non-null object
fechanacimiento    478311 non-null datetime64[ns]
sexo               504407 non-null object
edad               478302 non-null category
dtypes: category(1), datetime64[ns](1), object(2)
memory usage: 12.0+ MB


In [13]:
postulantes_genero_y_edad['edad'] = pd.cut(postulantes_genero_y_edad['edad'], range(0, 100, 5))

In [20]:
postulantes_genero_y_edad.head()

,idpostulante,fechanacimiento,sexo,edad
0,NM5M,1970-12-03,FEM,entre45y50
1,5awk,1962-12-04,FEM,entre55y60
2,ZaO5,1978-08-10,FEM,entre35y40
3,NdJl,1969-05-09,MASC,entre45y50
4,eo2p,1981-02-16,MASC,entre35y40


In [16]:
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype(str)

In [18]:
dic = {'(45, 50]': 'entre45y50','(55, 60]': 'entre55y60','(35, 40]': 'entre35y40'}

In [19]:
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].map(dic)

In [45]:
# Separa la gente postulada, luego los busco en el DF que junta todos los postulantes
# y me quedo con los opuestos, los no postulados.
# esto me va a servir para samplear
# GENTE NO POSTULADA
gente_no_postulada = postulantes_genero_y_edad[-postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_no_postulada = gente_no_postulada.reset_index(drop=True)

In [69]:
gente_no_postulada['sexo'].value_counts()

FEM           71992
MASC          67819
NO_DECLARA    15707
0.0               6
Name: sexo, dtype: int64

In [46]:
# Pocos datos con sexo invalido, los desecho ya que son los no postulados
gente_no_postulada = gente_no_postulada.drop(gente_no_postulada[gente_no_postulada['sexo'] == '0.0'].index)

In [47]:
# GENTE POSTULADA
gente_postulada = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_postulada = gente_postulada.reset_index(drop=True)

In [72]:
gente_postulada['sexo'].value_counts()

FEM           179213
MASC          160034
NO_DECLARA      9635
0.0                1
Name: sexo, dtype: int64

In [48]:
# Reemplazo el sexo invalido por NO_DECLARA
gente_postulada.loc[gente_postulada['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

In [49]:
# Dropeo los datos invalidos de los no postulados (se podria buscar alguna forma de salvarlos)
gente_no_postulada = gente_no_postulada.dropna()
gente_no_postulada = gente_no_postulada.reset_index(drop=True)
#gente_no_postulada['edad'] = gente_no_postulada['edad'].astype('int32')

In [50]:
# En un principio los relleno con la media de los postulantes
gente_postulada.loc[gente_postulada['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
gente_postulada['edad'] = gente_postulada['edad'].astype('int32')

In [77]:
gente_postulada.isnull().sum()

idpostulante           0
fechanacimiento    10074
sexo                   0
edad               10078
dtype: int64

# Funciones de utilidad

In [51]:
# fijarse de balancear el sample con 50% de postulados y no postulados

# Generador de sample
# el id tiene que estar en la columna 0
# Recibe los DFs de idpostulante e idaviso
def generador_de_sample_df(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = idpostulante.drop(columns=idpostulante.columns[1:])
    idaviso = idaviso.drop(columns=idaviso.columns[1:])
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [52]:
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [53]:
def borrar_tuplas_test_del_sample(sample,test_final):
    df_return = sample.merge(test_final,how='left',on=['idaviso','idpostulante'],indicator=True)
    df_return = df_return[df_return['_merge'] == 'left_only']
    df_return = df_return.drop('_merge',axis=1)
    return df_return

In [54]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

# Funciones feature

In [55]:
def feature_postulante_se_postulo_al_aviso(tupla_idaviso_idpostulante,postulaciones):
    df_return = tupla_idaviso_idpostulante.merge(postulaciones,on=['idaviso','idpostulante'],how='left')
    return df_return

In [56]:
def feature_genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
    return df_return

# Gente postulada

In [57]:
tupla_postulada = postulaciones.drop('fechapostulacion',axis=1)
tupla_postulada['y'] = 1
tupla_postulada = feature_genero_edad(tupla_postulada,gente_postulada)
tupla_postulada = tupla_postulada.drop('fechanacimiento',axis=1)
tupla_postulada = oneHotEncoding(tupla_postulada,'sexo')

In [58]:
tupla_postulada.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112257047,NM5M,1,48,1,0,0
1,1111920714,NM5M,1,48,1,0,0
2,1112346945,NM5M,1,48,1,0,0
3,1112345547,NM5M,1,48,1,0,0
4,1112237522,5awk,1,56,1,0,0


# Gente no postulada

In [59]:
sample = generador_de_sample_serie(avisos_detalle['idaviso'],gente_no_postulada['idpostulante'],100,100)
sample['y'] = 0
sample = feature_genero_edad(sample,gente_no_postulada)
sample = sample.drop('fechanacimiento',axis=1)
sample = oneHotEncoding(sample,'sexo')

In [87]:
sample.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112219832,xkd8jxY,0,"(10, 20]",1,0,0
1,1112219832,lD5XReM,0,"(30, 40]",0,1,0
2,1112219832,PmGQEME,0,"(20, 30]",0,1,0
3,1112219832,8MPpE8R,0,"(20, 30]",0,1,0
4,1112219832,zvarpmb,0,"(20, 30]",0,1,0


# Test Final

In [60]:
test_final = feature_genero_edad(test_final,gente_postulada.append(gente_no_postulada))

In [61]:
# Veo si hay datos nulos
test_final.isnull().sum()

idaviso               0
idpostulante          0
fechanacimiento    2838
sexo               2278
edad               2278
dtype: int64

In [62]:
# Salvo datos nulos
test_final.loc[test_final['sexo'].isnull(),'sexo'] = 'NO_DECLARA'
test_final.loc[test_final['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())

In [63]:
test_final.isnull().sum()

idaviso               0
idpostulante          0
fechanacimiento    2838
sexo                  0
edad                  0
dtype: int64

In [64]:
test_final.head()

,idaviso,idpostulante,fechanacimiento,sexo,edad
0,7392606M9ZQR,6M9ZQR,1976-03-29,FEM,42.0
1,7392606v1xdL,6v1xdL,1987-06-27,MASC,31.0
2,739260ezRKm9,ezRKm9,1982-02-17,FEM,36.0
3,7585801Q35ej,1Q35ej,1949-12-24,MASC,69.0
4,758580EAN4J6,EAN4J6,1986-05-27,FEM,32.0


In [66]:
test_final = test_final.drop(['idaviso','idpostulante','fechanacimiento'],axis=1)
test_final['edad'] = test_final['edad'].astype('int32')
test_final = oneHotEncoding(test_final,'sexo')

In [67]:
test_final.head()

,edad,FEM,MASC,NO_DECLARA
0,42,1,0,0
1,31,0,1,0
2,36,1,0,0
3,69,0,1,0
4,32,1,0,0


# ML

In [68]:
sample = sample.append(tupla_postulada.sample(n=1000000,random_state=0))

In [69]:
sample.head()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
0,1112390041,GNZ8JWK,0,55.0,0,1,0
1,1112390041,ZDlz5E1,0,21.0,1,0,0
2,1112390041,a0N4q,0,43.0,1,0,0
3,1112390041,ep41j9,0,32.0,0,1,0
4,1112390041,awOL1m,0,34.0,1,0,0


In [70]:
sample.tail()

,idaviso,idpostulante,y,edad,FEM,MASC,NO_DECLARA
4019134,1112457433,RzMAmXo,1,24.0,1,0,0
1441298,1112306596,ekOJ0L9,1,31.0,0,1,0
1290650,1112262555,PmJExxE,1,29.0,1,0,0
4598828,1112296411,4rPXrkM,1,25.0,0,1,0
6441257,1112355579,xkPYo6z,1,42.0,0,1,0


In [71]:
sample = sample.drop(['idaviso','idpostulante'],axis=1)

In [78]:
sample.head()

,y,edad,FEM,MASC,NO_DECLARA
0,0,8.0,0,1,0
1,0,2.0,1,0,0
2,0,6.0,1,0,0
3,0,4.0,0,1,0
4,0,4.0,1,0,0


In [98]:
sample.edad.value_counts()

3.0     517863
2.0     496996
4.0     348718
5.0     218517
6.0     143386
1.0     119622
7.0      75481
8.0      43707
9.0      23980
10.0      8684
11.0      1611
12.0      1128
13.0       305
0.0          2
Name: edad, dtype: int64

In [76]:
sample['edad'] = pd.cut(sample['edad'], range(0, 100, 5))
sample['edad'] = sample['edad'].astype(str)
dic = {'(10, 15]':0 , '(15, 20]':1 , '(20, 25]':2 , '(25, 30]':3 , '(30, 35]':4 , '(35, 40]':5 , '(40, 45]':6 , '(45, 50]':7 , '(50, 55]':8 , '(55, 60]':9 , '(60, 65]':10 , '(65, 70]':11 , '(70, 75]':12 , '(75, 80]':13 , '(80, 85]':14 , '(85, 90]':15 , '(95, 100]':16 , '(240, 245]':17}
sample['edad'] = sample['edad'].map(dic)

In [73]:
test_final = test_final.drop(['idaviso','idpostulante','fechanacimiento'],axis=1)

ValueError: labels ['idaviso' 'idpostulante' 'fechanacimiento'] not contained in axis

In [81]:
test_final.head()

,edad,FEM,MASC,NO_DECLARA
0,6.0,1,0,0
1,4.0,0,1,0
2,5.0,1,0,0
3,11.0,0,1,0
4,4.0,1,0,0


In [80]:
test_final['edad'] = pd.cut(test_final['edad'], range(0, 100, 5))
test_final['edad'] = test_final['edad'].astype(str)
test_final['edad'] = test_final['edad'].map(dic)

In [94]:
sample['edad'] = sample['edad'].fillna(2)

In [95]:
sample.isnull().sum()

y             0
edad          0
FEM           0
MASC          0
NO_DECLARA    0
dtype: int64

In [96]:
test_final['edad'] = test_final['edad'].fillna(2)

In [97]:
test_final.isnull().sum()

edad          0
FEM           0
MASC          0
NO_DECLARA    0
dtype: int64

In [82]:
#-----------------------------------------------------------------
# Modelo Random Forest Classifier
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

In [99]:
train_label = []
train_data = []

train_label = sample.iloc[:, 0]
train_data = sample.iloc[:, sample.columns != 'y']

In [100]:
ModeloRFC = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)

In [101]:
ModeloRFC.fit(train_data, train_label)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [102]:
result = ModeloRFC.predict(test_final)

In [103]:
result.size

100000

In [104]:
import csv

In [105]:
def write_submission(test_data, prediction, file_output):
    
    archivo_entrada = open(test_data)
    entrada_csv = csv.reader(archivo_entrada)
    next(entrada_csv, None)  # skip the headers

    archivo_salida = open(file_output, 'w')
    submit_csv = csv.writer(archivo_salida)
    submit_csv.writerow(['id', 'sepostulo'])

    for reg1, reg2  in zip(entrada_csv, prediction):
        linea = [reg1[0], round(reg2, 2)]
        submit_csv.writerow(linea)
    archivo_salida.close()

In [106]:
write_submission("../data/test_final_100k.csv", result, "../predicciones/RFC_edadPorRango.csv")

In [62]:
#-----------------------------------------------------------------
# Modelo GradientBoostingClassifier
#-----------------------------------------------------------------
from sklearn.ensemble import GradientBoostingClassifier

In [64]:
modeloGBR=GradientBoostingClassifier(n_estimators=150)

In [67]:
modeloGBR.fit(train_data, train_label)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=150,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [69]:
result = modeloGBR.predict(test_final)

In [70]:
result.size

100000

In [72]:
write_submission("../data/test_final_100k.csv", result, "../predicciones/GBC.csv")